# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
pip install py gmaps

In [9]:
# Dependencies and Setup
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests 
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [10]:
# Create a df of city data created in weatherpy
saved_city_path = "../output_data/cities.csv"

# Read the city data
city_df = pd.read_csv(saved_city_path)

# Drop any rows with null values
city_df = city_df.dropna()
len(city_df)


557

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["City latitude", "City longitude"]]
humidity  = city_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer( locations
                                , weights       = humidity 
                                , dissipating   = False
                                , max_intensity = 100
                                , point_radius  = 1
                                )


# Add layer
fig.add_layer(heat_layer)

# Display figure
print (fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow down the cities to fit weather conditions
city_reduced_df = city_df.loc[ ( (city_df["Max temperature"] > 50) 
                               & (city_df["Max temperature"] < 75)  
                               ) 
                               & (city_df["Wind speed"] < 10 ) 
                               & (city_df["Cloud coverage"] == 1)
                               , [ "City"           
                                 , "City country" 
                                 , "City latitude"  
                                 , "City longitude" ]
                               ].reset_index(drop=True) 
len(city_reduced_df)

1

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Store into variable named hotel_df.
hotel_df = city_reduced_df.rename(columns={  "City"           : "City"
                                          , "City country"   : "Country"
                                          , "City latitude"  : "Latitude"
                                          , "City longitude" : "Longitude"
                                          }
)
# Add columns for hotel name
hotel_df["Hotel Name"] = ""

In [22]:
#Set parameters to search for hotels within 5000 meters.
params = { "keyword" : "hotel"
         ,  "radius" : 5000
         ,  "type"   : "lodging"
         ,  "key"    : g_key
         }

# Save the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through hotel_df to search for the first hotel for each city 
for index, row in hotel_df.iterrows():
    
    # Save the Latitude and Longitude
    latitude  = row["Latitude"]
    longitude = row["Longitude"]
    
    # Set/add the location in the "params" dict
    params["location"] = f"{latitude}, {longitude}"
    
    # Assemble url and make API request and format the response into JSON
    city      = row["City"]
    print (f"Retrieving Results for index {index}: city {city}.")
    
    # Send request to Google Places API for each city's coordinates.
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in city {city} is {results[0]['name']}.")
        
        # Store the first Hotel result into the DataFrame.
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("-----------------------------------------------------------------------------------------")

# Display hotel_df
hotel_df

Retrieving Results for index 0: city Richards Bay.
Missing field/result... skipping.
-----------------------------------------------------------------------------------------


,City,Country,Latitude,Longitude,Hotel Name
0,Richards Bay,ZA,-28.783,32.0377,


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)

# Create symbol layer. hotel_info has the hotel name, city and country 
hotel_layer = gmaps.symbol_layer(  locations
                                 , fill_color='rgba(0, 150, 0, 0.4)'
                                 , stroke_color='rgba(0, 0, 150, 0.4)'
                                 , scale=2
                                 , info_box_content=hotel_info
)

#Plot markers on top of the heatmap.
fig = gmaps.figure()

fig.add_layer(markers)
fig.add_layer(hotel_layer)


# Display figure
print (fig)

Figure(layout=FigureLayout(height='420px'))
